In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import requests

In [5]:
def load_data(file_path, url_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
    else:
        d,i,offset_size = [],0,100000
        k = offset_size
        while k == offset_size:
            if i > 10:
                break
            r=requests.get(
                url = url_path, 
                params={
                    '$limit':offset_size, 
                    '$offset':offset_size*i
                }
            )
    
            as_df = pd.DataFrame(r.json())
            k = len(as_df)
            i += 1
            d.append(as_df)
    
        df = pd.concat(d)
        df.to_csv(file_path)
        df.head(10)
    return df

In [7]:
tree_census_df =load_data('data/tree_census_15.csv', 'https://data.cityofnewyork.us/resource/5rq2-4hqu.json')

In [9]:
air_quality_df = load_data('data/Air_Quality_20241206.csv', 'https://data.cityofnewyork.us/resource/c3uy-2p5r.json')

In [11]:
green_streets_df = load_data('data/Greenstreets.csv', 'https://data.cityofnewyork.us/resource/mk9u-qu7i.json')

In [13]:
property_valuation_df = load_data('data/Property_Valuation_and_Assessment_Data_20241125.csv', 'https://data.cityofnewyork.us/resource/yjxr-fw8i.json')

# Tree Census Data Sanitation 

In [15]:
tree_census_df.shape

(683788, 43)

# Air Quality Data Sanitation

In [17]:
air_quality_df.head()

,Unnamed: 0,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
0,0,827080,386,Ozone (O3),Mean,ppb,UHF34,104.0,Pelham - Throgs Neck,Summer 2022,2022-06-01T00:00:00.000,33.3
1,1,827061,386,Ozone (O3),Mean,ppb,UHF34,405.0,Ridgewood - Forest Hills,Summer 2022,2022-06-01T00:00:00.000,34.2
2,2,827067,386,Ozone (O3),Mean,ppb,UHF34,302.0,Central Harlem - Morningside Heights,Summer 2022,2022-06-01T00:00:00.000,30.9
3,3,827081,386,Ozone (O3),Mean,ppb,UHF34,103.0,Fordham - Bronx Pk,Summer 2022,2022-06-01T00:00:00.000,31.7
4,4,825967,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,104.0,Pelham - Throgs Neck,Summer 2022,2022-06-01T00:00:00.000,12.0


In [19]:
air_quality_df = air_quality_df.dropna(subset=['geo_join_id'])

In [21]:
air_quality_df['geo_join_id'] = air_quality_df['geo_join_id'].astype('int64')

In [27]:
air_quality_df['name'].unique()

array(['Ozone (O3)', 'Nitrogen dioxide (NO2)', 'Fine particles (PM 2.5)',
       'Annual vehicle miles traveled (cars)',
       'Annual vehicle miles traveled (trucks)',
       'Annual vehicle miles traveled',
       'Respiratory hospitalizations due to PM2.5 (age 20+)',
       'Asthma emergency department visits due to PM2.5',
       'Asthma emergency departments visits due to Ozone',
       'Cardiovascular hospitalizations due to PM2.5 (age 40+)',
       'Cardiac and respiratory deaths due to Ozone',
       'Asthma hospitalizations due to Ozone', 'Deaths due to PM2.5',
       'Boiler Emissions- Total PM2.5 Emissions',
       'Boiler Emissions- Total SO2 Emissions',
       'Boiler Emissions- Total NOx Emissions',
       'Outdoor Air Toxics - Formaldehyde',
       'Outdoor Air Toxics - Benzene'], dtype=object)

In [29]:
air_quality_df = air_quality_df[air_quality_df['name'].str.contains('Outdoor Air Toxics') | air_quality_df['name'].str.contains('Boiler Emissions') ]

In [39]:
simplified_tree_df = tree_census_df[['the_geom', 'zipcode', 'zip_city', 'cb_num', 'borocode', 'boroname', 'boro_ct', 'state']]

In [49]:
simplified_tree_df = simplified_tree_df[(simplified_tree_df['zipcode'] >= 10000) & (simplified_tree_df['zipcode']<11500)]

In [55]:
simplified_tree_df.shape

(670351, 8)

In [57]:
simplified_tree_df = simplified_tree_df.drop_duplicates(subset=['cb_num'])

In [59]:
simplified_tree_df.shape

(59, 8)

In [71]:
air_quality_geo_df =  air_quality_df.merge(simplified_tree_df, left_on='geo_join_id', right_on='cb_num')

In [75]:
air_quality_geo_df.head()

,Unnamed: 0,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value,the_geom,zipcode,zip_city,cb_num,borocode,boroname,boro_ct,state
0,8151,179825,641,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,406,Fresh Meadows,2015,2015-01-01T00:00:00.000,0.4,"{'type': 'Point', 'coordinates': [-73.84421521...",11375,Forest Hills,406,4,Queens,4073900,New York
1,8157,179783,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,104,Pelham - Throgs Neck,2015,2015-01-01T00:00:00.000,2.8,"{'type': 'Point', 'coordinates': [-73.98729652...",10019,New York,104,1,Manhattan,1012700,New York
2,8158,179773,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,404,Bayside - Little Neck,2015,2015-01-01T00:00:00.000,0.9,"{'type': 'Point', 'coordinates': [-73.86529991...",11373,Elmhurst,404,4,Queens,4044302,New York
3,8159,179722,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,410,Rockaways,2015,2015-01-01T00:00:00.000,6.1,"{'type': 'Point', 'coordinates': [-73.84344476...",11414,Howard Beach,410,4,Queens,4089200,New York
4,8160,179792,640,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,206,Borough Park,2015,2015-01-01T00:00:00.000,1.1,"{'type': 'Point', 'coordinates': [-73.89338225...",10457,Bronx,206,2,Bronx,2037504,New York


# Green Streets Data Sanitation

In [25]:
green_streets_df.shape

(2753, 26)

# Property Valuation Data Sanitation

In [26]:
property_valuation_df.shape

(1100000, 45)